# 데이터 업로드

In [1]:
import pandas as pd
import numpy as np
import re
import os

import pickle
import shutil
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
from google.colab import drive
drive.mount('/content/drive')
current_path = os.getcwd()
print(current_path)

Mounted at /content/drive
/content


In [3]:
path=r'/content/drive/MyDrive/기업프로젝트/머신러닝/데이터/testset.csv'
df=pd.read_csv(path,index_col=0)
path=r'/content/drive/MyDrive/기업프로젝트/머신러닝/데이터/Hynix_train2.csv'
df2=pd.read_csv(path, index_col=0)

from sklearn.model_selection import train_test_split
train,test= train_test_split(df2, test_size=0.2,random_state=1234)
train,valid= train_test_split(train, test_size=0.25,random_state=1234)

#클래스화

In [4]:
class Preprocessor :
    default_path = '/content/drive/MyDrive/기업프로젝트/머신러닝/데이터/Pickle'

    def __init__(self):
        pass

    def time_preprocessing(self,df):
        df.set_index(keys="ID", inplace=True)
        df.drop(columns="x204",inplace=True)
        for idx,col in enumerate(df.columns.to_list()[188:1454]):
            if df[col].dtype == "object":
                df[col] = pd.to_datetime(df[col])
        datatmp = df.columns.to_list()[188:1454]
        ts_data = df.select_dtypes("datetime")
        for idx in ts_data.index:
            ts_data.sort_values(by=idx,axis=1, inplace=True)

        columns = ts_data.columns
        num_columns = len(columns)
        for i in range(1, num_columns):
            column_diff = pd.to_datetime(df.iloc[:, i]) - pd.to_datetime(ts_data.iloc[:, i-1])
            column_diff_hours = column_diff.dt.total_seconds() / 3600
            ts_data.iloc[:, i] = column_diff_hours

        for i in range(len(ts_data.columns)-1):
            ts_data.iloc[:, i] = ts_data.iloc[:, i+1]
        result = []
        for idx,col in enumerate(datatmp):
            if df[col].dtype == "<M8[ns]":
                cur = int(col[1:])
                i = idx
                tmp = []
                while i > 0:
                    i -= 1
                    next = datatmp[i]
                    if df[next].dtype == "<M8[ns]":
                        break
                    else:
                        tmp.append(next)
                        tmp.sort()
                result.append((col,tmp))
        ts_final = []
        for elem in ts_data.columns:
            for target,content in result:
                if elem == target:
                    ts_final.extend(content)
                    ts_final.append(target)

        ts_final = df[ts_final]
        front = df.loc[:,:"x193"]
        back = df.loc[:,"x1461":]
        final = pd.concat([front, ts_final, back], axis = 1)
        ts_data.drop(columns=ts_data.columns[-1], inplace=True)

        for col in ts_data.columns:
            if col in final.columns:
                final[col] = ts_data[col]
        final.drop(['x197'], axis=1, inplace=True)
        df = final
        return df





    def na_ratio_preprocessing_train(self,train):
        na_percentage = train.isna().sum() / len(train)
        high_na_columns = na_percentage[na_percentage >= 0.9].index
        train=train.drop(high_na_columns, axis=1,inplace=False)
        with open(Preprocessor.default_path + '/high_na_columns.pkl', 'wb') as f:
          pickle.dump(high_na_columns, f)
        return train

    def na_ratio_preprocessing_Rtest(self,Rtest):
        with open(Preprocessor.default_path + '/high_na_columns.pkl', 'rb') as f:
          high_na_columns = pickle.load(f)
        Rtest=Rtest.drop(high_na_columns, axis=1,inplace=False)
        return Rtest





    def correlation_preprocessing_train(self,train):
        numeric_columns = train.select_dtypes(include=['int64', 'float64'])
        correlation_with_target = numeric_columns.corr()['Y']
        columns_with_low_correlation = correlation_with_target[correlation_with_target.abs() < 0.04].index
        train.drop(columns_with_low_correlation, axis=1, inplace=True)
        with open(Preprocessor.default_path+'/columns_with_low_correlation.pkl', 'wb') as f:
            pickle.dump(columns_with_low_correlation, f)
        return train

    def correlation_preprocessing_Rtest(self,Rtest):
        with open(Preprocessor.default_path+'/columns_with_low_correlation.pkl', 'rb') as f:
          columns_with_low_correlation = pickle.load(f)
        Rtest.drop(columns_with_low_correlation, axis=1, inplace=True)
        return Rtest





    def replace_outliers_median_preprocessing_train(self,train , threshold=1.5):
        all_lower_bound={}
        all_upper_bound={}
        all_col_median={}
        for col in train.columns:
            Q1 = train[col].quantile(0.25)
            Q3 = train[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - threshold * IQR
            all_lower_bound[col]=lower_bound
            upper_bound = Q3 + threshold * IQR
            all_upper_bound[col]=upper_bound

            train_outliers = train[col][(train[col] < lower_bound) | (train[col] > upper_bound)]
            col_median = train[col].median()
            all_col_median[col]=col_median
            train.loc[train_outliers.index, col] = col_median
        with open(Preprocessor.default_path+'/all_lower_bound.pkl', 'wb') as f:
            pickle.dump(all_lower_bound, f)
        with open(Preprocessor.default_path+'/all_upper_bound.pkl', 'wb') as f:
            pickle.dump(all_upper_bound, f)
        with open(Preprocessor.default_path+'/all_col_median.pkl', 'wb') as f:
            pickle.dump(all_col_median, f)
        return train

    def replace_outliers_median_preprocessing_Rtest(self,Rtest, threshold=1.5):
        with open(Preprocessor.default_path+'/all_lower_bound.pkl', 'rb') as f:
            all_lower_bound = pickle.load(f)
        with open(Preprocessor.default_path+'/all_upper_bound.pkl', 'rb') as f:
            all_upper_bound = pickle.load(f)
        with open(Preprocessor.default_path+'/all_col_median.pkl', 'rb') as f:
            all_col_median = pickle.load(f)

        for col in Rtest.columns:
            Rtest_outliers = Rtest[col][(Rtest[col] < all_lower_bound[col]) | (Rtest[col] > all_upper_bound[col])]
            Rtest.loc[Rtest_outliers.index, col] = all_col_median[col]
        return Rtest






    def fillna_mean_preprocessing_train(self,train):
        col_means = {}
        use_col = []

        for col in train.columns:
            col_mean = train[col][~train[col].isnull()].mean()
            if pd.notna(col_mean):
                train[col].fillna(col_mean, inplace=True)
                use_col.append(col)
                if train[col].nunique() == 1:
                  use_col.remove(col)
            col_means[col] = col_mean
        with open(Preprocessor.default_path+'/col_means.pkl', 'wb') as f:
            pickle.dump(col_means, f)
        with open(Preprocessor.default_path+'/use_col.pkl', 'wb') as f:
            pickle.dump(use_col, f)
        return train[use_col]

    def fillna_mean_preprocessing_Rtest(self,Rtest):
        with open(Preprocessor.default_path+'/col_means.pkl', 'rb') as f:
            col_means = pickle.load(f)
        with open(Preprocessor.default_path+'/use_col.pkl', 'rb') as f:
            use_col = pickle.load(f)

        for col in Rtest.columns:
            if col in use_col:
                Rtest[col].fillna(col_means[col], inplace=True)
        return Rtest[use_col]





    def scale_preprocessing_train(self,train):
        scaler = MinMaxScaler()
        strain = scaler.fit_transform(train)
        with open(Preprocessor.default_path+'/scaler.pkl', 'wb') as f:
          pickle.dump(scaler, f)
        train = pd.DataFrame(strain, columns=train.columns,index=train.index)
        return train

    def scale_preprocessing_Rtest(self,Rtest):
        with open(Preprocessor.default_path+'/scaler.pkl', 'rb') as f:
            scaler = pickle.load(f)
        sRtest = scaler.transform(Rtest)
        Rtest = pd.DataFrame(sRtest, columns=Rtest.columns,index=Rtest.index)
        return Rtest





    def preprocessing_train(self, train):
      train = self.na_ratio_preprocessing_train(train)
      train = self.correlation_preprocessing_train(train)
      train = train.drop(columns=['Y'])
      train = self.replace_outliers_median_preprocessing_train(train)
      train = self.fillna_mean_preprocessing_train(train)
      train = self.scale_preprocessing_train(train)
      return train




    def preprocessing_Rtest(self,df):
        Rtest = self.time_preprocessing(df)
        Rtest = self.na_ratio_preprocessing_Rtest(Rtest)
        Rtest = self.correlation_preprocessing_Rtest(Rtest)
        Rtest = self.replace_outliers_median_preprocessing_Rtest(Rtest)
        Rtest = self.fillna_mean_preprocessing_Rtest(Rtest)
        Rtest = self.scale_preprocessing_Rtest(Rtest)
        return Rtest

In [6]:
# preprocess = Preprocessor()
# default_path = '/content/drive/MyDrive/기업프로젝트/머신러닝/데이터/Pickle/'
# with open(default_path+"Preprocessor", "wb") as f:
#     pickle.dump(preprocess, f)

In [ ]:
default_path = '/content/drive/MyDrive/기업프로젝트/머신러닝/데이터/Pickle/'
with open(default_path+"Preprocessor", "rb") as f:
     preprocess = pickle.load(f)

In [ ]:
train = preprocess.preprocessing_train(train)

In [ ]:
Rtest = preprocess.preprocessing_Rtest(df)

# 모델 예측 결과

In [ ]:
best_model = load_model('/content/drive/MyDrive/기업프로젝트/머신러닝/데이터/ML_best_model')
predictions = predict_model(best_model, data=Rtest)['prediction_label']*100